In [1]:
import gensim
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np
import re
import csv

import pandas as pd
import pprint

import string
import nltk


from IPython.display import HTML


In [2]:
pathToDatasets = '../datasets/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
word_vectors = api.load("glove-wiki-gigaword-100")
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize



senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


NUMBER_OF_ALTERNATIVES = 7
TWEET_START = 50
NUM_OF_TWEETS = 50

/home/ubuntu/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

## Utility Code

In [5]:
def cleanAndTokenizeText(text):
    text = text.lower()
    newString = ""
    for char in text:
        if char not in string.punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    return possibleReplacements

def getPOSTags(tweet):
    tags = nltk.pos_tag(tweet)
    return(tags)

## NLP Utility Code

In [6]:
def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    return possibleReplacements


def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    truePOSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = truePOSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]

        if(str(newTag) == str(mainTag)):
            print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    return validWords
        

In [7]:
def getAlternativeSentences(tweet, sentimentOfTweet):
    userInputTokens = cleanAndTokenizeText(tweet)
    
   
    alternativeStrings = []
    for ind,word in enumerate(userInputTokens):
        
        score = senty.polarity_scores(word)['compound'] # get the aggregated score!
        newUserTokens = userInputTokens[:]
        
        if(score != 0.0):
            replacements = listReplacements(word)       
            replacements = posApprovedReplacements(replacements[:], newUserTokens[:], ind)
            if(replacements == []):
                continue          
            print("Word changed: {0}".format(word));
            for newWord in replacements:
                
                newUserTokens[ind] = cstr(" <i>[{0}]</i> ".format(newWord), 'blue');
                newString = ' '.join(newUserTokens)
                sentimentOfNewString = senty.polarity_scores(newString)['compound']
             
            
                if(sentimentOfNewString == 0):
                    continue
                
                alternativeStrings.append(newString)
    return alternativeStrings;

## Main Cells

In [8]:
def runThroughTweets():

    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=NUM_OF_TWEETS, skiprows=TWEET_START)

    tweets = tweets_df.values

    
    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
       
        newStrings = getAlternativeSentences(tweet, mainSentiment)
        if(newStrings == [] or newStrings == None):
            continue
        print("\nNUM({2}) {0}:{1}\n".format(tweet,mainSentiment, counter))
        for alteredTweet in newStrings:
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            if((sentimentOfNewString) >= (mainSentiment)):
                display(HTML(cstr("{0}: {1}".format(alteredTweet,sentimentOfNewString),'green')))
            else:
                display(HTML(cstr("{0}: {1}".format(alteredTweet,sentimentOfNewString),'red')))
    
    
runThroughTweets()

Word broken[NN] replaced with breaking[NN]
Word broken[NN] replaced with broke[NN]
Word broken[NN] replaced with apart[NN]
Word broken[NN] replaced with neck[NN]
Word changed: broken
Word promise[NN] replaced with pledge[NN]
Word promise[NN] replaced with commitment[NN]
Word promise[NN] replaced with hope[NN]
Word promise[NN] replaced with give[NN]
Word promise[NN] replaced with desire[NN]
Word changed: promise

NUM(0) broadband plan 'a massive broken promise'  via  still waiting for broadband we are :-0.2023



Word wow[JJ] replaced with c'mon[JJ]
Word wow[JJ] replaced with hey[JJ]
Word wow[JJ] replaced with gosh[JJ]
Word wow[JJ] replaced with whew[JJ]
Word changed: wow
Word friends[NNS] replaced with parents[NNS]
Word friends[NNS] replaced with relatives[NNS]
Word friends[NNS] replaced with others[NNS]
Word changed: friends

NUM(1)  wow tons of replies from you may have to unfollow so i can see my friends' tweets you're scrolling the feed a lot. :0.7845



Word forget[VBP] replaced with remember[VBP]
Word forget[VBP] replaced with tell[VBP]
Word forget[VBP] replaced with imagine[VBP]
Word forget[VBP] replaced with know[VBP]
Word changed: forget
Word hug[NN] replaced with hugs[NN]
Word hug[NN] replaced with kiss[NN]
Word hug[NN] replaced with hugged[NN]
Word hug[NN] replaced with goodbye[NN]
Word changed: hug
Word want[VBP] replaced with do[VBP]
Word want[VBP] replaced with know[VBP]
Word want[VBP] replaced with get[VBP]
Word changed: want
Word assets[NNS] replaced with investments[NNS]
Word assets[NNS] replaced with funds[NNS]
Word assets[NNS] replaced with debts[NNS]
Word changed: assets
Word sorry[VBP] replaced with 'm[VBP]
Word sorry[VBP] replaced with glad[VBP]
Word sorry[VBP] replaced with sad[VBP]
Word sorry[VBP] replaced with regret[VBP]
Word changed: sorry

NUM(5)  not sure what they are only that they are pos! as much as i want to i dont think can trade away company assets sorry andy! :-0.2134



Word hate[NN] replaced with anyone[NN]
Word hate[NN] replaced with crime[NN]
Word changed: hate
Word sad[JJ] replaced with awful[JJ]
Word sad[JJ] replaced with tragic[JJ]
Word sad[JJ] replaced with horrible[JJ]
Word sad[JJ] replaced with happy[JJ]
Word sad[JJ] replaced with poignant[JJ]
Word changed: sad
Word feeling[NN] replaced with feel[NN]
Word feeling[NN] replaced with felt[NN]
Word feeling[NN] replaced with sense[NN]
Word changed: feeling

NUM(7) i have a sad feeling that dallas is not going to show up  i gotta say though you'd think more shows would use music from the game. mmm:-0.3818



Word glad[JJ] replaced with thankful[JJ]
Word glad[JJ] replaced with pleased[JJ]
Word glad[JJ] replaced with happy[JJ]
Word changed: glad
Word well[RB] replaced with so[RB]
Word well[RB] replaced with even[RB]
Word changed: well

NUM(9) where did u move to?  i thought u were already in sd. ?? hmmm. random u found me. glad to hear yer doing well.:0.6848



...

Word poor[JJ] replaced with bad[JJ]
Word poor[JJ] replaced with low[JJ]
Word changed: poor
Word pray[NN] replaced with prayed[NN]
Word pray[NN] replaced with god[NN]
Word pray[NN] replaced with mourn[NN]
Word pray[NN] replaced with prayer[NN]
Word changed: pray
Word please[VB] replaced with wish[VB]
Word please[VB] replaced with call[VB]
Word please[VB] replaced with forget[VB]
Word changed: please
Word threatening[VBG] replaced with causing[VBG]
Word changed: threatening
Word party[NN] replaced with opposition[NN]
Word party[NN] replaced with coalition[NN]
Word party[NN] replaced with leader[NN]
Word party[NN] replaced with election[NN]
Word changed: party
Word jerk[NN] replaced with snatch[NN]
Word jerk[NN] replaced with kgs[NN]
Word jerk[NN] replaced with lifter[NN]
Word jerk[NN] replaced with goetschl[NN]
Word changed: jerk

NUM(43) pray for me please the ex is threatening to start sh** at my/our babies 1st birthday party. what a jerk. and i still have a headache :0.128



Word enjoy[VB] replaced with prefer[VB]
Word enjoy[VB] replaced with feel[VB]
Word changed: enjoy
Word problems[NNS] replaced with difficulties[NNS]
Word problems[NNS] replaced with troubles[NNS]
Word problems[NNS] replaced with concerns[NNS]
Word changed: problems

NUM(44)  hmm   do u really enjoy being with him ? if the problems are too constants u should think things more  find someone ulike:0.2006



Word sick[JJ] replaced with ill[JJ]
Word sick[JJ] replaced with pregnant[JJ]
Word sick[JJ] replaced with elderly[JJ]
Word sick[JJ] replaced with infected[JJ]
Word sick[JJ] replaced with tired[JJ]
Word changed: sick
Word party[NN] replaced with opposition[NN]
Word party[NN] replaced with coalition[NN]
Word party[NN] replaced with leader[NN]
Word party[NN] replaced with election[NN]
Word party[NN] replaced with candidate[NN]
Word changed: party
Word sadly[RB] replaced with thankfully[RB]
Word sadly[RB] replaced with terribly[RB]
Word sadly[RB] replaced with frankly[RB]
Word sadly[RB] replaced with curiously[RB]
Word sadly[RB] replaced with painfully[RB]
Word sadly[RB] replaced with strangely[RB]
Word changed: sadly
Word well[RB] replaced with so[RB]
Word well[RB] replaced with even[RB]
Word changed: well

NUM(46) so ryleegrace...wana go steve's party or not?? sadly since its easter i wnt b able 2 do much  but ohh well.....:0.4359



Word bad[JJ] replaced with good[JJ]
Word changed: bad

NUM(47) hey i actually won one of my bracket pools! too bad it wasn't the one for money :0.126



Word bad[JJ] replaced with good[JJ]
Word changed: bad
Word favorite[JJ] replaced with favourite[JJ]
Word favorite[JJ] replaced with popular[JJ]
Word favorite[JJ] replaced with famous[JJ]
Word changed: favorite
Word lose[VBP] replaced with get[VBP]
Word lose[VBP] replaced with want[VBP]
Word changed: lose
Word good[JJ] replaced with sure[JJ]
Word changed: good

NUM(49) a bad nite for the favorite teams: astros and spartans lose.  the nite out with t.w. was good.:-0.0772

